In [ ]:
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime

In [ ]:
numbers = [****]

In [ ]:
numbersMissed = [***]

In [ ]:
numberMissed2 = [***]

In [ ]:
with open('data5.txt') as f:
   read_data = f.read()

In [ ]:
soup = BeautifulSoup(read_data)
tables = soup.find_all('tr', {'class': 'item'}) #Получаем все таблицы с таблицу с параметрами

In [ ]:
tableResult = np.empty(shape = (1,13), dtype = str)

for t in tables:
    soup = BeautifulSoup(str(t)) #Первая машина в таблице
    tablesInfo = soup.find_all('td')
    tablesInfoArray = np.char.replace(
                                      np.char.replace(
                                                      np.char.replace(np.array(list(map(str, np.array(tablesInfo, dtype=object)))), '<td>\n', ''), 
                                                                                                                        '</td>', ''), 
                                                                                                                '  ', '')
    try:
        for i in [4, 5, 6, 12]:
            soup = BeautifulSoup(str(tablesInfoArray[i]))
            toChange = soup.find_all('div', {'class': 'mobile-hidden'})
            tablesInfoArray[i] = np.char.replace(np.char.replace(toChange[0].contents[0], ' ', ''), '\n', '')
        tableResult = np.vstack((tableResult, tablesInfoArray))
    except Exception as err:
        error = tablesInfo
        print(err)
        notErrPrev = prev
    prev = tablesInfo

In [ ]:
result = pd.DataFrame(tableResult[1:], columns = ['Полное наименование', 'Сокращенное наименование', 'ОГРН', 
                                                  'Номер разрешения', 'Дата внесения реестровой записи о выдаче разрешения', 
                                                  'Дата начала действия разрешения', 'Срок действия разрешения', 
                                                  'Серия и номер бланка разрешения', 'Марка автомобиля', 'Модель автомобиля', 
                                                  'Государственный номер', 'Признак аннулирования', 'Дата аннулирования'])

In [ ]:
mask = result['Номер разрешения'].str.contains(r'Укажите номер разрешения', na=True)
result.loc[mask, 'Номер разрешения'] = result.loc[mask, 'Номер разрешения'].replace('Укажите номер разрешения', '-')
for c in ['Полное наименование', 'Сокращенное наименование', 'Серия и номер бланка разрешения', 
          'Марка автомобиля', 'Модель автомобиля', 'Государственный номер', 'Признак аннулирования']:
    result[c] = result[c].astype(str)
for c in ['ОГРН', 'Номер разрешения']:
    result[c] = result[c].astype(str)
for c in ['Дата внесения реестровой записи о выдаче разрешения', 'Дата начала действия разрешения', 
          'Срок действия разрешения', 'Дата аннулирования']:
    result.loc[~result[c].isin(['']), c] = pd.to_datetime(result.loc[~result[c].isin(['']), c], format = '%d.%m.%Y').dt.date

In [ ]:
mask = result['Государственный номер'].str.contains(r' ', na=True)
result.loc[mask, 'Государственный номер'] = result.loc[mask, 'Государственный номер'].replace(to_replace=' ', value='', regex=True)

In [ ]:
mask = result['Государственный номер'].str.contains(r'RUS', na=True)
result.loc[mask, 'Государственный номер'] = result.loc[mask, 'Государственный номер'].replace(to_replace='RUS', value='', regex=True)

In [ ]:
#result = result.convert_dtypes()

In [ ]:
#pd.to_datetime(result[~result['Дата внесения реестровой записи о выдаче разрешения'].isin([''])]['Дата внесения реестровой записи о выдаче разрешения'], format = '%d.%m.%Y')

In [ ]:
result.tail(5)

In [ ]:
#~((result['Признак аннулирования'] == 'Да') & (result['Государственный номер'].isin(numbers))) &

In [ ]:
np.setdiff1d(numberMissed2, result[result['Государственный номер'].isin(numberMissed2)]['Государственный номер'].unique())

In [ ]:
len(result[result['Государственный номер'].isin(numberMissed2)]['Государственный номер'].unique())

In [ ]:
dfBad = result[(result['Признак аннулирования'] == 'Нет') & 
       (pd.to_datetime(result['Срок действия разрешения'], format = '%Y-%m-%d').dt.date > datetime.date(2022, 6, 16))]

In [ ]:
dfBad

In [ ]:
#dfBad.to_csv('taxiBadFiltered.csv', header = True)

In [ ]:
#result.to_csv('taxiAll.csv', header = True)

In [ ]:
dfBad.to_excel('taxiBadFiltered3Stage.xlsx', header = True)

In [ ]:
result.to_excel('taxiAll3Stage.xlsx', header = True)